In [ ]:
import torch
import torchvision
from torch import nn
import os
import matplotlib.pyplot as plt
from torchvision import transforms
import random
import pandas as pd
import seaborn as sns
from PIL import Image
from pathlib import Path
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
%matplotlib inline

### Dataset Link https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset/data

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
image_path=r"D:\00_google_classroom\machinelearning.ai\02_transfer_learning\images\images"
train_dir=os.path.join(image_path,"train")
test_dir=os.path.join(image_path,"validation")

In [ ]:
# Get list of classes and image paths
classes = os.listdir(train_dir)
all_images = []

for cls in classes:
    class_path = os.path.join(train_dir, cls)
    images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
    all_images.extend(images)

# Randomly select 25 images
random_images = random.sample(all_images, 25)

# Plot the 5x5 grid
plt.figure(figsize=(12, 12))

for i, img_path in enumerate(random_images):
    img = Image.open(img_path)
    plt.subplot(5, 5, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(os.path.basename(os.path.dirname(img_path)))  # show class label

plt.tight_layout()
plt.show()

In [ ]:
# Get list of classes and image paths
classes = os.listdir(test_dir)
all_images = []

for cls in classes:
    class_path = os.path.join(test_dir, cls)
    images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
    all_images.extend(images)

# Randomly select 25 images
random_images = random.sample(all_images, 25)

# Plot the 5x5 grid
plt.figure(figsize=(12, 12))

for i, img_path in enumerate(random_images):
    img = Image.open(img_path)
    plt.subplot(5, 5, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(os.path.basename(os.path.dirname(img_path)))  # show class label

plt.tight_layout()
plt.show()

In [ ]:
class_counts = {}

for class_name in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_name)
    if os.path.isdir(class_path):
        count = len(os.listdir(class_path))
        class_counts[class_name] = count

# Convert to DataFrame
df_counts = pd.DataFrame(list(class_counts.items()), columns=['Class', 'Image Count'])

# Plot using Seaborn
plt.figure(figsize=(15, 6))
sns.barplot(data=df_counts, x='Class', y='Image Count', palette='Dark2')
plt.title("Number of Images per Class in Test Directory")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Lambda(lambda image: image.convert("RGB")),  # always 3 channels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])
image_path=Path(image_path)
image_path_list = list(image_path.glob("*/*/*.jpg"))

In [ ]:
def plot_transformed_images(image_paths, transform, n=5, seed=42):
    random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f)
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            transformed_image = transform(f).permute(1, 2, 0)
            ax[1].imshow(transformed_image)
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

plot_transformed_images(image_path_list, transform=data_transform, n=5)

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
class_dict = train_data.class_to_idx
class_dict

In [ ]:
len(train_data), len(test_data)

In [ ]:
img, label = train_data[0][0], train_data[0][1]
print(f"Image tensor:\n{img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=8,  # how many samples per batch?
    num_workers=0,  # how many subprocesses to use for data loading? (higher = more)
    shuffle=True    # shuffle the data?
)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=8,
    num_workers=0,
    shuffle=False   # don't usually need to shuffle testing data
)

train_dataloader, test_dataloader

In [ ]:
feature_extractor = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
feature_extractor.fc = nn.Identity()
feature_extractor = feature_extractor.to(device)
feature_extractor.eval()

def extract_features(dataset):
    features = []
    labels = []
    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(device)
            feats = feature_extractor(imgs)
            features.append(feats.cpu().numpy())
            labels.append(lbls.cpu().numpy())
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

train_features, train_labels = extract_features(train_data)
test_features, test_labels = extract_features(test_data)

In [ ]:
svm_clf = SVC(kernel='linear', probability=True)
svm_clf.fit(train_features, train_labels)

In [ ]:
test_preds = svm_clf.predict(test_features)
print("Classification Report:\n", classification_report(test_labels, test_preds, target_names=class_names))
cm = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
def predict_image(image_path, feature_extractor, svm_clf, class_names):
    image = Image.open(image_path)
    image = data_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        feature = feature_extractor(image).cpu().numpy()
    pred = svm_clf.predict(feature)[0]
    prob = svm_clf.predict_proba(feature)[0][pred]
    plt.imshow(Image.open(image_path))
    plt.title(f"Prediction: {class_names[pred]} ({prob*100:.2f}%)")
    plt.axis('off')
    plt.show()
    return class_names[pred], prob

sample_image_path = r"D:\00_google_classroom\machinelearning.ai\02_transfer_learning\images\images\validation\disgust\1115.jpg"
predict_image(sample_image_path, feature_extractor, svm_clf, class_names)

In [ ]:
import joblib
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "facial_expression_svm.joblib")
joblib.dump(svm_clf, save_path)
print(f"SVM model saved successfully in {save_path}")

In [ ]:
import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)